In [2]:
import os
import pandas as pd
from pandas import isnull

# Reading data
## Master.csv

In [3]:
master = pd.read_csv(os.path.join("data", "Master.csv"))
master.head()

,playerID,coachID,hofID,firstName,lastName,nameNote,nameGiven,nameNick,height,weight,...,birthDay,birthCountry,birthState,birthCity,deathYear,deathMon,deathDay,deathCountry,deathState,deathCity
0,aaltoan01,NaN,NaN,Antti,Aalto,NaN,Antti,NaN,73.0,210.0,...,4.0,Finland,NaN,Lappeenranta,NaN,NaN,NaN,NaN,NaN,NaN
1,abbeybr01,NaN,NaN,Bruce,Abbey,NaN,Bruce,NaN,73.0,185.0,...,18.0,Canada,ON,Toronto,NaN,NaN,NaN,NaN,NaN,NaN
2,abbotge01,NaN,NaN,George,Abbott,NaN,George Henry,Preacher,67.0,153.0,...,3.0,Canada,ON,Synenham,NaN,NaN,NaN,NaN,NaN,NaN
3,abbotre01,NaN,NaN,Reg,Abbott,NaN,Reginald Stewart,NaN,71.0,164.0,...,4.0,Canada,MB,Winnipeg,NaN,NaN,NaN,NaN,NaN,NaN
4,abdelju01,NaN,NaN,Justin,Abdelkader,NaN,NaN,NaN,73.0,195.0,...,25.0,USA,MI,Muskegon,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
master.shape

(7761, 31)

In [5]:
master.columns

Index(['playerID', 'coachID', 'hofID', 'firstName', 'lastName', 'nameNote',
       'nameGiven', 'nameNick', 'height', 'weight', 'shootCatch', 'legendsID',
       'ihdbID', 'hrefID', 'firstNHL', 'lastNHL', 'firstWHA', 'lastWHA', 'pos',
       'birthYear', 'birthMon', 'birthDay', 'birthCountry', 'birthState',
       'birthCity', 'deathYear', 'deathMon', 'deathDay', 'deathCountry',
       'deathState', 'deathCity'],
      dtype='object')

In [6]:
# Braces let us structure the code in a readable way
(master['playerID']
    .pipe(isnull)
    .value_counts())
# .pipe is like Unix. Passes the previous to the next function as an argument

False    7520
True      241
Name: playerID, dtype: int64

In [7]:
# This is an alternative to the above:
isnull(master['playerID']).value_counts()
# 241 missing data rows

False    7520
True      241
Name: playerID, dtype: int64

In [8]:
# Removing null entries
master_orig = master.copy()
master.dropna(subset=['playerID'], inplace=True)
master.shape

(7520, 31)

In [9]:
master = master.dropna(subset=['firstNHL', 'lastNHL'], how='all')
# how=all ensures both entries need to be null

In [10]:
# filtering
master = master.loc[master['lastNHL']>=1980]
master.shape

(4627, 31)

In [11]:
# Removing unwanted columns the hard way
keeping = ['playerID', 'firstName', 'lastName',
          'pos', 'birthYear', 'birthMon', 'birthDay',
          'birthCountry', 'birthState', 'birthCity']
master[keeping].head()
master.filter(keeping).head() #another way to show

,playerID,firstName,lastName,pos,birthYear,birthMon,birthDay,birthCountry,birthState,birthCity
0,aaltoan01,Antti,Aalto,C,1975.0,3.0,4.0,Finland,NaN,Lappeenranta
4,abdelju01,Justin,Abdelkader,L,1987.0,2.0,25.0,USA,MI,Muskegon
9,abidra01,Ramzi,Abid,L,1980.0,3.0,24.0,Canada,QC,Montreal
11,abrahth01,Thommy,Abrahamsson,D,1947.0,4.0,12.0,Sweden,NaN,Leksand
14,actonke01,Keith,Acton,C,1958.0,4.0,15.0,Canada,ON,Stouffville


In [12]:
# Advanced and easy filtering with regex
master = master.filter(regex="(playerID|pos|^birth)|(Name$)")
master.columns

Index(['playerID', 'firstName', 'lastName', 'pos', 'birthYear', 'birthMon',
       'birthDay', 'birthCountry', 'birthState', 'birthCity'],
      dtype='object')

In [13]:
# Verify memory consumption
def mem_mib(df):
    print("{0:.2f} MiB".format(
        df.memory_usage().sum() / (1024*1024)
    ))
mem_mib(master)
mem_mib(master_orig)

0.39 MiB
1.84 MiB


In [14]:
#Analyze a columns value counts
master['pos'].value_counts()

D      1418
C      1037
L       848
R       832
G       463
F        28
L/C       1
Name: pos, dtype: int64

In [15]:
# Categories
pd.Categorical(master['pos'])

[C, L, L, D, C, ..., R, L, L, C, D]
Length: 4627
Categories (7, object): [C, D, F, G, L, L/C, R]

In [16]:
# Improve memory consumption
def make_categorical(df, col_name):
    df.loc[:, col_name] = pd.Categorical(df[col_name])
make_categorical(master, 'pos')
make_categorical(master, 'birthCountry')
make_categorical(master, 'birthState')
mem_mib(master)

0.30 MiB


In [17]:
# Since we`ve dropped many rows, the default index is now random. Solve by:
master = master.set_index('playerID')
master.head()

,firstName,lastName,pos,birthYear,birthMon,birthDay,birthCountry,birthState,birthCity
playerID,,,,,,,,,
aaltoan01,Antti,Aalto,C,1975.0,3.0,4.0,Finland,NaN,Lappeenranta
abdelju01,Justin,Abdelkader,L,1987.0,2.0,25.0,USA,MI,Muskegon
abidra01,Ramzi,Abid,L,1980.0,3.0,24.0,Canada,QC,Montreal
abrahth01,Thommy,Abrahamsson,D,1947.0,4.0,12.0,Sweden,NaN,Leksand
actonke01,Keith,Acton,C,1958.0,4.0,15.0,Canada,ON,Stouffville


In [18]:
# Save for later use:
master.to_pickle('master.pickle')

## Scoring.csv

In [20]:
scoring = pd.read_csv(os.path.join("data", "Scoring.csv"))
mem_mib(scoring)
scoring.shape
scoring.head()

10.87 MiB


,playerID,year,stint,tmID,lgID,pos,GP,G,A,Pts,...,PostA,PostPts,PostPIM,Post+/-,PostPPG,PostPPA,PostSHG,PostSHA,PostGWG,PostSOG
0,aaltoan01,1997,1,ANA,NHL,C,3.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,aaltoan01,1998,1,ANA,NHL,C,73.0,3.0,5.0,8.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,aaltoan01,1999,1,ANA,NHL,C,63.0,7.0,11.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,aaltoan01,2000,1,ANA,NHL,C,12.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,abbeybr01,1975,1,CIN,WHA,D,17.0,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
def recent_nhl_only(df):
    return df[(df['lgID']=='NHL') & 
              (df['year'] >= 1980)]
scoring = recent_nhl_only(scoring)
scoring.shape

(28616, 31)

In [23]:
scoring.columns

Index(['playerID', 'year', 'stint', 'tmID', 'lgID', 'pos', 'GP', 'G', 'A',
       'Pts', 'PIM', '+/-', 'PPG', 'PPA', 'SHG', 'SHA', 'GWG', 'GTG', 'SOG',
       'PostGP', 'PostG', 'PostA', 'PostPts', 'PostPIM', 'Post+/-', 'PostPPG',
       'PostPPA', 'PostSHG', 'PostSHA', 'PostGWG', 'PostSOG'],
      dtype='object')

In [26]:
scoring = scoring.filter(regex="^(?!(Post|PP|SH)).*")
scoring.columns

Index(['playerID', 'year', 'stint', 'tmID', 'lgID', 'pos', 'GP', 'G', 'A',
       'Pts', 'PIM', '+/-', 'GWG', 'GTG', 'SOG'],
      dtype='object')

In [27]:
scoring.head()

,playerID,year,stint,tmID,lgID,pos,GP,G,A,Pts,PIM,+/-,GWG,GTG,SOG
0,aaltoan01,1997,1,ANA,NHL,C,3.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,1.0
1,aaltoan01,1998,1,ANA,NHL,C,73.0,3.0,5.0,8.0,24.0,-12.0,0.0,0.0,61.0
2,aaltoan01,1999,1,ANA,NHL,C,63.0,7.0,11.0,18.0,26.0,-13.0,1.0,0.0,102.0
3,aaltoan01,2000,1,ANA,NHL,C,12.0,1.0,1.0,2.0,2.0,1.0,0.0,0.0,18.0
7,abdelju01,2007,1,DET,NHL,L,2.0,0.0,0.0,0.0,2.0,0.0,0.0,NaN,6.0


In [28]:
# Filter further by column index; Remember: iloc is for number based indexing and loc is of label based.
scoring = scoring.iloc[:, [0,1,3,6,7,8,9,14]]
scoring.columns

Index(['playerID', 'year', 'tmID', 'GP', 'G', 'A', 'Pts', 'SOG'], dtype='object')

In [29]:
make_categorical(scoring, 'tmID')
scoring.head()

,playerID,year,tmID,GP,G,A,Pts,SOG
0,aaltoan01,1997,ANA,3.0,0.0,0.0,0.0,1.0
1,aaltoan01,1998,ANA,73.0,3.0,5.0,8.0,61.0
2,aaltoan01,1999,ANA,63.0,7.0,11.0,18.0,102.0
3,aaltoan01,2000,ANA,12.0,1.0,1.0,2.0,18.0
7,abdelju01,2007,DET,2.0,0.0,0.0,0.0,6.0


In [30]:
# reindex
scoring.reset_index(drop=True, inplace=True)
scoring.head()

,playerID,year,tmID,GP,G,A,Pts,SOG
0,aaltoan01,1997,ANA,3.0,0.0,0.0,0.0,1.0
1,aaltoan01,1998,ANA,73.0,3.0,5.0,8.0,61.0
2,aaltoan01,1999,ANA,63.0,7.0,11.0,18.0,102.0
3,aaltoan01,2000,ANA,12.0,1.0,1.0,2.0,18.0
4,abdelju01,2007,DET,2.0,0.0,0.0,0.0,6.0


In [31]:
scoring.to_pickle('scoring.pickle')

## Teams.csv

In [32]:
teams = pd.read_csv(os.path.join('data', 'Teams.csv'))
teams.shape

(1519, 27)

In [33]:
teams.columns

Index(['year', 'lgID', 'tmID', 'franchID', 'confID', 'divID', 'rank',
       'playoff', 'G', 'W', 'L', 'T', 'OTL', 'Pts', 'SoW', 'SoL', 'GF', 'GA',
       'name', 'PIM', 'BenchMinor', 'PPG', 'PPC', 'SHA', 'PKG', 'PKC', 'SHF'],
      dtype='object')

In [34]:
teams = recent_nhl_only(teams)
teams = teams[['year', 'tmID', 'name']]
teams.head()

,year,tmID,name
727,1980,BOS,Boston Bruins
728,1980,BUF,Buffalo Sabres
729,1980,CAL,Calgary Flames
730,1980,CHI,Chicago Black Hawks
731,1980,COR,Colorado Rockies


In [35]:
# CHeck number of unique values per column
teams.nunique()

year    31
tmID    37
name    37
dtype: int64

In [44]:
make_categorical(teams, 'tmID')
# We do not categorize name since it may hinder our capacity to play with strings

In [45]:
teams.to_pickle('teams.pickle')

# TeamSplits.csv

In [46]:
team_splits = pd.read_csv(os.path.join('data', 'TeamSplits.csv'))
team_splits.shape

(1519, 43)

In [47]:
team_splits = recent_nhl_only(team_splits)
team_splits.columns

Index(['year', 'lgID', 'tmID', 'hW', 'hL', 'hT', 'hOTL', 'rW', 'rL', 'rT',
       'rOTL', 'SepW', 'SepL', 'SepT', 'SepOL', 'OctW', 'OctL', 'OctT',
       'OctOL', 'NovW', 'NovL', 'NovT', 'NovOL', 'DecW', 'DecL', 'DecT',
       'DecOL', 'JanW', 'JanL', 'JanT', 'JanOL', 'FebW', 'FebL', 'FebT',
       'FebOL', 'MarW', 'MarL', 'MarT', 'MarOL', 'AprW', 'AprL', 'AprT',
       'AprOL'],
      dtype='object')

In [48]:
cols_to_drop = team_splits.columns[3:11]
team_splits = team_splits.drop(columns=cols_to_drop)
team_splits.columns

Index(['year', 'lgID', 'tmID', 'SepW', 'SepL', 'SepT', 'SepOL', 'OctW', 'OctL',
       'OctT', 'OctOL', 'NovW', 'NovL', 'NovT', 'NovOL', 'DecW', 'DecL',
       'DecT', 'DecOL', 'JanW', 'JanL', 'JanT', 'JanOL', 'FebW', 'FebL',
       'FebT', 'FebOL', 'MarW', 'MarL', 'MarT', 'MarOL', 'AprW', 'AprL',
       'AprT', 'AprOL'],
      dtype='object')

In [49]:
team_splits = team_splits.drop(columns='lgID')

In [50]:
make_categorical(team_splits, 'tmID')
team_splits.to_pickle('team_splits.pickle')

# Joins

Sample use case using pd.merge
```python
pd.merge(df1, df2,
        on='playerID')
        
pd.merge(df1, df2,
        left_on='playerID',
        right_on='plID')        
```

In [51]:
# Importing
master = pd.read_pickle('master.pickle')
scoring = pd.read_pickle('scoring.pickle')
teams = pd.read_pickle('teams.pickle')
team_splits = pd.read_pickle('team_splits.pickle')

In [53]:
master.head(2)

,firstName,lastName,pos,birthYear,birthMon,birthDay,birthCountry,birthState,birthCity
playerID,,,,,,,,,
aaltoan01,Antti,Aalto,C,1975.0,3.0,4.0,Finland,NaN,Lappeenranta
abdelju01,Justin,Abdelkader,L,1987.0,2.0,25.0,USA,MI,Muskegon


In [54]:
scoring.head(2)

,playerID,year,tmID,GP,G,A,Pts,SOG
0,aaltoan01,1997,ANA,3.0,0.0,0.0,0.0,1.0
1,aaltoan01,1998,ANA,73.0,3.0,5.0,8.0,61.0


In [55]:
# Merge on index and column
pd.merge(master, scoring, left_index=True, right_on='playerID').head()

,firstName,lastName,pos,birthYear,birthMon,birthDay,birthCountry,birthState,birthCity,playerID,year,tmID,GP,G,A,Pts,SOG
0,Antti,Aalto,C,1975.0,3.0,4.0,Finland,NaN,Lappeenranta,aaltoan01,1997,ANA,3.0,0.0,0.0,0.0,1.0
1,Antti,Aalto,C,1975.0,3.0,4.0,Finland,NaN,Lappeenranta,aaltoan01,1998,ANA,73.0,3.0,5.0,8.0,61.0
2,Antti,Aalto,C,1975.0,3.0,4.0,Finland,NaN,Lappeenranta,aaltoan01,1999,ANA,63.0,7.0,11.0,18.0,102.0
3,Antti,Aalto,C,1975.0,3.0,4.0,Finland,NaN,Lappeenranta,aaltoan01,2000,ANA,12.0,1.0,1.0,2.0,18.0
4,Justin,Abdelkader,L,1987.0,2.0,25.0,USA,MI,Muskegon,abdelju01,2007,DET,2.0,0.0,0.0,0.0,6.0


In [56]:
scoring.index

RangeIndex(start=0, stop=28616, step=1)

In [57]:
# Which index does the merge keep? The one from the table we join on column!
scoring.index + 3

RangeIndex(start=3, stop=28619, step=1)

In [59]:
scoring.index = scoring.index + 3

In [61]:
# Observer the index started with the one we changed from the right table
pd.merge(master, scoring, left_index=True, right_on='playerID').head()

,firstName,lastName,pos,birthYear,birthMon,birthDay,birthCountry,birthState,birthCity,playerID,year,tmID,GP,G,A,Pts,SOG
3,Antti,Aalto,C,1975.0,3.0,4.0,Finland,NaN,Lappeenranta,aaltoan01,1997,ANA,3.0,0.0,0.0,0.0,1.0
4,Antti,Aalto,C,1975.0,3.0,4.0,Finland,NaN,Lappeenranta,aaltoan01,1998,ANA,73.0,3.0,5.0,8.0,61.0
5,Antti,Aalto,C,1975.0,3.0,4.0,Finland,NaN,Lappeenranta,aaltoan01,1999,ANA,63.0,7.0,11.0,18.0,102.0
6,Antti,Aalto,C,1975.0,3.0,4.0,Finland,NaN,Lappeenranta,aaltoan01,2000,ANA,12.0,1.0,1.0,2.0,18.0
7,Justin,Abdelkader,L,1987.0,2.0,25.0,USA,MI,Muskegon,abdelju01,2007,DET,2.0,0.0,0.0,0.0,6.0


In [64]:
# Inner join by default
# Compare entries by join type
print(
    pd.merge(master, scoring, left_index=True, right_on='playerID').shape,
    pd.merge(master, scoring, left_index=True, right_on='playerID', how='right').shape
)

(28616, 17) (28616, 17)


In [65]:
# Drop some random row and check once again
master2 = master.drop(master.sample(5).index)
print(
    pd.merge(master2, scoring, left_index=True, right_on='playerID').shape,
    pd.merge(master2, scoring, left_index=True, right_on='playerID', how='right').shape
)

(28580, 17) (28616, 17)


In [66]:
merged = pd.merge(master2, scoring, left_index=True, right_on='playerID', how='right', indicator=True)
merged.head()

,firstName,lastName,pos,birthYear,birthMon,birthDay,birthCountry,birthState,birthCity,playerID,year,tmID,GP,G,A,Pts,SOG,_merge
3,Antti,Aalto,C,1975.0,3.0,4.0,Finland,NaN,Lappeenranta,aaltoan01,1997,ANA,3.0,0.0,0.0,0.0,1.0,both
4,Antti,Aalto,C,1975.0,3.0,4.0,Finland,NaN,Lappeenranta,aaltoan01,1998,ANA,73.0,3.0,5.0,8.0,61.0,both
5,Antti,Aalto,C,1975.0,3.0,4.0,Finland,NaN,Lappeenranta,aaltoan01,1999,ANA,63.0,7.0,11.0,18.0,102.0,both
6,Antti,Aalto,C,1975.0,3.0,4.0,Finland,NaN,Lappeenranta,aaltoan01,2000,ANA,12.0,1.0,1.0,2.0,18.0,both
7,Justin,Abdelkader,L,1987.0,2.0,25.0,USA,MI,Muskegon,abdelju01,2007,DET,2.0,0.0,0.0,0.0,6.0,both


In [67]:
merged['_merge'].value_counts()

both          28580
right_only       36
left_only         0
Name: _merge, dtype: int64

In [68]:
merged[merged['_merge']=='right_only'].head()

,firstName,lastName,pos,birthYear,birthMon,birthDay,birthCountry,birthState,birthCity,playerID,year,tmID,GP,G,A,Pts,SOG,_merge
6133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drakeda01,1992,DET,72.0,18.0,26.0,44.0,89.0,right_only
6134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drakeda01,1993,DET,47.0,10.0,22.0,32.0,78.0,right_only
6135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drakeda01,1993,WIN,15.0,3.0,5.0,8.0,34.0,right_only
6136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drakeda01,1994,WIN,43.0,8.0,18.0,26.0,66.0,right_only
6137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drakeda01,1995,WIN,69.0,19.0,20.0,39.0,121.0,right_only


In [69]:
scoring2 = scoring.drop(scoring.sample(1000).index)

In [70]:
merged = pd.merge(master2, scoring2, left_index=True, right_on='playerID', how='outer', indicator=True)

In [71]:
merged[(merged['_merge']=='left_only') |
       (merged['_merge']=='right_only')
      ].sample(3)

,firstName,lastName,pos,birthYear,birthMon,birthDay,birthCountry,birthState,birthCity,playerID,year,tmID,GP,G,A,Pts,SOG,_merge
NaN,Peter,Helander,D,1951.0,12.0,4.0,Sweden,NaN,Stockholm,helanpe01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
6147.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drakeda01,2006.0,STL,60.0,6.0,6.0,12.0,74.0,right_only
9156.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,guenina01,2008.0,PHI,1.0,0.0,0.0,0.0,0.0,right_only


In [73]:
# Alternative to the above:
merged[merged['_merge'].str.endswith('only')].sample(3)

,firstName,lastName,pos,birthYear,birthMon,birthDay,birthCountry,birthState,birthCity,playerID,year,tmID,GP,G,A,Pts,SOG,_merge
6148.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drakeda01,2007.0,DET,65.0,3.0,3.0,6.0,46.0,right_only
6142.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drakeda01,2000.0,STL,82.0,12.0,29.0,41.0,142.0,right_only
NaN,Jonas,Nordqvist,C,1982.0,4.0,26.0,Sweden,NaN,Leksand,nordqjo01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [75]:
# Validate that the join was 1 to many:
pd.merge(master, scoring,
         left_index=True,
         right_on='playerID',
         validate='1:m').head()

,firstName,lastName,pos,birthYear,birthMon,birthDay,birthCountry,birthState,birthCity,playerID,year,tmID,GP,G,A,Pts,SOG
3,Antti,Aalto,C,1975.0,3.0,4.0,Finland,NaN,Lappeenranta,aaltoan01,1997,ANA,3.0,0.0,0.0,0.0,1.0
4,Antti,Aalto,C,1975.0,3.0,4.0,Finland,NaN,Lappeenranta,aaltoan01,1998,ANA,73.0,3.0,5.0,8.0,61.0
5,Antti,Aalto,C,1975.0,3.0,4.0,Finland,NaN,Lappeenranta,aaltoan01,1999,ANA,63.0,7.0,11.0,18.0,102.0
6,Antti,Aalto,C,1975.0,3.0,4.0,Finland,NaN,Lappeenranta,aaltoan01,2000,ANA,12.0,1.0,1.0,2.0,18.0
7,Justin,Abdelkader,L,1987.0,2.0,25.0,USA,MI,Muskegon,abdelju01,2007,DET,2.0,0.0,0.0,0.0,6.0


In [76]:
# Sample of exception
try:
    pd.merge(master, scoring,
             left_index=True,
             right_on='playerID',
             validate='1:1').head()
except Exception as e:
    print(e)

Merge keys are not unique in right dataset; not a one-to-one merge


In [79]:
merged = merged.filter(regex="^(?!(birth)).*") # drop birth data
merged.head()

,firstName,lastName,pos,playerID,year,tmID,GP,G,A,Pts,SOG,_merge
3.0,Antti,Aalto,C,aaltoan01,1997.0,ANA,3.0,0.0,0.0,0.0,1.0,both
4.0,Antti,Aalto,C,aaltoan01,1998.0,ANA,73.0,3.0,5.0,8.0,61.0,both
5.0,Antti,Aalto,C,aaltoan01,1999.0,ANA,63.0,7.0,11.0,18.0,102.0,both
7.0,Justin,Abdelkader,L,abdelju01,2007.0,DET,2.0,0.0,0.0,0.0,6.0,both
8.0,Justin,Abdelkader,L,abdelju01,2008.0,DET,2.0,0.0,0.0,0.0,2.0,both


In [80]:
merged.to_pickle('scoring_merged.pickle')